In [1]:
#This block generates the hartree-fock wavefunction for iodine atom
import os,sys,numpy
from pyscf import gto
from pyscf import scf
mol = gto.M(
    atom = 'I 0 0 0',
    basis = 'sto-3g',
    verbose = 4,
    symmetry = 1,
    spin = 1)
myhf = scf.RHF(mol)
myhf.irrep_nelec = {'A1g':(7,7), 'A1u':(4,3), 'E1ux':(4,4), 'E1uy':(4,4), 'E2gx':(2,2), 'E2gy':(2,2), 'E1gx':(2,2), 'E1gy':(2,2)}
myhf.kernel()

System: ('Linux', 'shci', '4.4.0-124-generic', '#148-Ubuntu SMP Wed May 2 13:00:18 UTC 2018', 'x86_64', 'x86_64')  Threads 8
Python 2.7.14 |Anaconda custom (64-bit)| (default, Oct 16 2017, 17:29:19) 
[GCC 7.2.0]
numpy 1.13.3  scipy 0.19.1
Date: Thu May 17 11:29:55 2018
PySCF version 1.4.3
PySCF path  /home/xubo/Documents/pyscf/pyscf
GIT HEAD      a153fc27dc6a4d5287189baf7417757404c4a2b4

[INPUT] VERBOSE 4
[INPUT] num atoms = 1
[INPUT] num electrons = 53
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry 1 subgroup None
[INPUT]  1 I      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr
[INPUT] ---------------- BASIS SET ---------------- 
[INPUT] l, kappa, [nprim/nctr], expnt,             c_1 c_2 ...
[INPUT] I
[INPUT] 0    0    [3    /1   ]  6035.183623       0.1543289673
                                1099.316231       0.5353281423
                                297.5178737       0.4446345422
[INPU

Warn: Ipython shell catchs sys.args


cycle= 1 E= -6850.67573594438  delta_E= -22.4  |g|= 0.0528  |ddm|= 1.14
cycle= 2 E= -6850.67624557599  delta_E= -0.00051  |g|= 0.0026  |ddm|= 0.0199
cycle= 3 E= -6850.67624699529  delta_E= -1.42e-06  |g|= 8.04e-06  |ddm|= 0.00109
cycle= 4 E= -6850.67624699529  delta_E= -1.82e-12  |g|= 4.54e-09  |ddm|= 4.74e-07
Extra cycle  E= -6850.67624699529  delta_E= 9.09e-13  |g|= 1.68e-10  |ddm|= 1.28e-09
converged SCF energy = -6850.67624699529


-6850.676246995291

In [2]:
# Prepare the 3 pairs of states with Px, Py or Pz single occupied
# as 3 seperate shci object for the further state average calculation
from pyscf.shciscf import shci, settings
shci1 = shci.SHCI(mol)
shci1.irrep_nelec = {'A1g':(1,1),'A1u':(1,1),'E1ux':(1,1),'E1uy':(1,0)}
shci1.prefix = os.environ['PWD']+'/shci1'
shci1.mpiprefix = 'mpirun -np 4'
shci1.nPTiter = 0
shci1.sweep_iter = [0]
shci1.sweep_epsilon = [1e-8]

shci2 = shci.SHCI(mol)
shci2.irrep_nelec = {'A1g':(1,1),'A1u':(1,1),'E1ux':(1,0),'E1uy':(1,1)}
shci2.prefix = os.environ['PWD']+'/shci2'
shci2.mpiprefix = 'mpirun -np 4'
shci2.nPTiter = 0
shci2.sweep_iter = [0]
shci2.sweep_epsilon = [1e-8]

shci3 = shci.SHCI(mol)
shci3.irrep_nelec = {'A1g':(1,1),'A1u':(1,0),'E1ux':(1,1),'E1uy':(1,1)}
shci3.prefix = os.environ['PWD']+'/shci3'
shci3.mpiprefix = 'mpirun -np 4'
shci3.nPTiter = 0
shci3.sweep_iter = [0]
shci3.sweep_epsilon = [1e-8]



Warning

Modules in the "future" directory (dmrgscf, fciqmcscf, shciscf, icmspt, xianci)
have been moved to pyscf/pyscf directory.  You can still import these modules.
from the "future" directory, and they work the same as before.

To avoid name conflicts with python built-in module "future", this directory
will be deleted in future release.



In [3]:
# Performs the state average calculation
from pyscf import mcscf
mycas = shci.SHCISCF(myhf, 4, 7)
mycas.mo_coeff = 1.*myhf.mo_coeff
mcscf.state_average_mix_(mycas, [shci1, shci2, shci3], numpy.ones(3)/3)
mo = mcscf.addons.sort_mo_by_irrep(mycas, mycas.mo_coeff, {'A1g':1, 'A1u':1, 'E1ux':1, 'E1uy':1})
mycas.kernel(mo)

ncore for each irreps {'A1u': 3, 'E1gy': 2, 'E1gx': 2, 'E2gx': 2, 'E2gy': 2, 'E1uy': 3, 'E1ux': 3, 'A1g': 6}
ncas for each irreps {'A1u': 1, 'E1uy': 1, 'A1g': 1, 'E1ux': 1}
(0-based) caslst = [23 24 25 26]

******** <class 'pyscf.mcscf.mc1step_symm.CASSCF'> flags ********
CAS (4e+3e, 4o), ncore = 23, nvir = 0
max_cycle_macro = 50
max_cycle_micro = 4
conv_tol = 1e-07
conv_tol_grad = None
orbital rotation max_stepsize = 0.02
augmented hessian ah_max_cycle = 30
augmented hessian ah_conv_tol = 1e-12
augmented hessian ah_linear dependence = 1e-14
augmented hessian ah_level shift = 0
augmented hessian ah_start_tol = 2.5
augmented hessian ah_start_cycle = 3
augmented hessian ah_grad_trust_region = 3
kf_trust_region = 3
kf_interval = 4
ci_response_space = 4
ci_grad_trust_region = 3
with_dep4 0
natorb = False
canonicalization = True
sorting_mo_energy = False
chkfile = /home/xubo/scratch/tmpTFiMM3
max_memory 4000 MB (current use 100 MB)
internal_rotation = False
******** SHCI flags ********
exec

(-6850.6762469952928,
 -10.264066582593841,
 [0, 0, 0],
 NPArrayWithTag([[  9.95731647e-01,  -4.53301126e-01,   0.00000000e+00,
                    0.00000000e+00,   0.00000000e+00,   2.49649165e-01,
                    0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                    1.89415279e-07,   0.00000000e+00,   0.00000000e+00,
                    0.00000000e+00,   0.00000000e+00,  -1.01308015e-01,
                    0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                    2.82561390e-05,   0.00000000e+00,   0.00000000e+00,
                    0.00000000e+00,   0.00000000e+00,   2.42720548e-02,
                    0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
                 [  1.00890915e-02,   1.04282918e+00,   0.00000000e+00,
                    0.00000000e+00,   0.00000000e+00,  -7.25400356e-01,
                    0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                   -5.68293029e-07,   0.00000000e+00,   0.00000000e+00,
       

In [4]:
print ''
print '********Writing SOC integrals*******'
print ''

os.system('cp FCIDUMP FCIDUMP_bp')
sys.stdout.flush()
shci.writeSOCIntegrals(mycas, pictureChange1e='bp', pictureChange2e='bp')


********Writing SOC integrals*******



In [5]:
# Performs the one-step soc calculation with Dice
# ZDice2 is the binary of Dice that works with complex value integrals
os.system('ZDice2 soc.dat > soc.out')

0

In [6]:
# Clean up all the temporary files
os.system('rm -rf shci* tmp* *.bkp')

0